In [1]:
require 'rbplotly'

true

# ベクトルの定義

In [2]:
class MyVector < Array
  def *(scalar)
    self.map{ |elem| elem * scalar }.to_v
  end
  
  def +(other)
    self.zip(other).map { |self_elem, other_elem| self_elem + other_elem }.to_v
  end
  
  def -(other)
    self + (other * -1)
  end
  
  def /(scalar)
    self.map { |elem| elem / scalar }.to_v
  end
  
  def norm
    Math.sqrt(self.sum { |elem| elem**2 })
  end
end

class Array
  def to_v
    MyVector.new(self)
  end
=begin
  def sum
    if block_given?
      self.map { |elem| yield(elem) }.reduce(:+)
    else
      self.reduce(:+)
    end
  end
=end
end

:to_v

# 傾きを出す関数f

$$
\frac{d {\bf y} }{dt} = f({\bf y})
$$


In [3]:
def f(y)
  r1, r2, v1, v2 = y
  return [v1, v2, v2, -v1].to_v
end

:f

# calculate関数

dt、max_t(tの最大値)を引数として与え、数値解析アルゴリズムをブロックで与えると、ts, rc, ra, errを返す。

- 数値解析アルゴリズムのブロック
    - tsとdtを与えると、ysを返すブロック
- ts
    - $t=0$から、dtを一つづつ足していった配列を返す(0, dt, 2dt, 3dt, ... < max_t)。
- rc
    - 数値解の配列。$t=0, dt, 2dt, 3dt ...$に対応している。
- ra
    - 解析解の配列。$t=0, dt, 2dt, 3dt ...$に対応している。
- err
    - $\left| {\rm r_a}(t) - {\rm r_c}(t) \right|$の配列。$t=0, dt, 2dt, 3dt ...$に対応している。

In [4]:
def calculate(dt, max_t, &ys_block)
  ts = (0..Float::INFINITY).lazy.map{ |i| i * dt }.take_while { |t| t < max_t }.to_a

  ys = ys_block.call(ts, dt)

  rc = [ ys.transpose[0], ys.transpose[1] ]
  ra = [ ts.map{ |t| - Math.cos(t) }, ts.map{ |t| Math.sin(t) } ]
  err = (rc.transpose).zip(ra.transpose).map { |_rc, _ra| (_rc.to_v - _ra.to_v).norm }

  return ts, rc, ra, err
end

:calculate

# all_calculations関数

period(周期)、max_tを引数として与え、数値解析アルゴリズムをブロックで与えると、ts, rc, ra, err, p_errを返す。

内部的にはcalculate関数を呼ぶが、そのときのdtは$\frac{{\rm period}}{64}$である。

ts, rc, ra, errは、calculateと同じ値となる。

p_errは、課題3.2における$p$の値と$E_r$の配列である。([[$p$, $E_r$],[$p$,$E_r$],[$p$,$E_r$]])

In [5]:
def all_calculations(period, max_t, &ys_block)
  p_err = (3..30).map do |p|
    _, _, _, err = calculate(period * 2**(-p), period, &ys_block)
    [p, err.max]
  end
  
  ts, rc, ra, err = calculate(period/64, max_t, &ys_block)
  
  return ts, rc, ra, err, p_err
end

:all_calculations

In [6]:
def draw_graphs(ts, rc, ra, err, p_err)
  trace_rc = { x: rc[0], y: rc[1], name: 'rc' }
  trace_ra = { x: ra[0], y: ra[1], name: 'ra' }

  Plotly::Plot.new(data: [trace_rc, trace_ra], layout: { width: 500, height: 500 }).show

  trace_error = { x: ts, y: err }
  Plotly::Plot.new(data: [trace_error]).show
  
  trace_p_error = { x: p_err.transpose[0], y: p_err.transpose[1] }
  Plotly::Plot.new(data: [trace_p_error], layout: { yaxis: { type: :log } }).show
end

:draw_graphs

In [7]:
period = 6.4

6.4

In [8]:
ts, rc, ra, err, p_err = all_calculations(period, period*5) do |ts, dt|
  ys = []
  ts.each_with_index do |t, i|
    if i == 0
      yn =[-1.0, 0, 0, 1.0].to_v # 初期値
    else
      yn = ys.last + f(ys.last) * dt
    end
    ys.push(yn)
  end
  ys
end
nil

NoMethodError: undefined method `sum' for [0.0, 0.0]:MyVector

In [9]:
draw_graphs(ts, rc, ra, err, p_err)

NoMethodError: undefined method `[]' for nil:NilClass

In [10]:
ts, rc, ra, err, p_err = all_calculations(period, period*5) do |ts, dt|
  ys = []
  ts.each_with_index do |t, i|
    if i == 0
      yn = [-1.0, 0, 0, 1.0].to_v
    else
      ys_1 = ys.last + f(ys.last) * dt
      yn = ys.last + (f(ys_1) + f(ys.last)) * (1.0/2) * dt
    end
    ys.push(yn)
  end
  ys
end
nil

NoMethodError: undefined method `sum' for [0.0, 0.0]:MyVector

In [11]:
draw_graphs(ts, rc, ra, err, p_err)

NoMethodError: undefined method `[]' for nil:NilClass

In [12]:
ts, rc, ra, err, p_err = all_calculations(period, period*5) do |ts, dt|
  ys = []
  ts.each_with_index do |t, i|
    if i == 0
      yn = [-1.0, 0, 0, 1.0].to_v
    else
      k1 = f(ys.last)
      k2 = f(ys.last + k1 * (dt/2))
      k3 = f(ys.last + k2 * (dt/2))
      k4 = f(ys.last + k3 * dt)
      k = (k1 + k2 * 2.0 + k3 * 2.0 + k4)/6.to_f
      yn = ys.last + k * dt
    end
    ys.push(yn)
  end
  ys
end
nil

NoMethodError: undefined method `sum' for [0.0, 0.0]:MyVector

In [13]:
draw_graphs(ts, rc, ra, err, p_err)

NoMethodError: undefined method `[]' for nil:NilClass